# [**Interactive visualization of COVID19 outbreak in India**](https://github.com/MoadComputer/covid19-visualization/blob/master/examples/COVID19_India.ipynb)
## Author: [Dr. Rahul Remanan](https://www.linkedin.com/in/rahulremanan/), CEO [Moad Computer](https://moad.computer)
### Contact: rahul@moad.computer

This notebook creates an interactive visualization of the statewise COVID19 outbreak in India using GeoPandas and Bokeh.
## **[Run this notebook in Google Colab](https://colab.research.google.com/github/MoadComputer/covid19-visualization/blob/master/examples/COVID19_India.ipynb)**

# Import libraries

In [1]:
import os, re, sys, math, json, bokeh, geopandas, numpy as np, pandas as pd
from scipy.interpolate import interp1d 
from bokeh.io.doc import curdoc
from bokeh.layouts import layout
from bokeh.plotting import figure
from bokeh.models.glyphs import Text
from bokeh.application import Application
from bokeh.models.callbacks import CustomJS
from bokeh.plotting import show as plt_show
from bokeh.palettes import brewer,OrRd,YlGn
from bokeh.models.widgets import Button,Select
from bokeh.tile_providers import Vendors,get_provider
from bokeh.io import output_notebook,show,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.layouts import widgetbox,row,column,gridplot
from bokeh.models import ColumnDataSource,Slider,HoverTool,Select,Div,        \
                         Range1d,WMTSTileSource,BoxZoomTool,TapTool,Panel,Tabs
from bokeh.models import GeoJSONDataSource,LinearColorMapper,ColorBar,        \
                         NumeralTickFormatter, LinearAxis,Grid,Label,Band,    \
                         Legend,LegendItem

In [2]:
DATA_URL = 'https://raw.githubusercontent.com/MoadComputer/covid19-visualization/main/data'

# Reading map

In [3]:
India_statewise=geopandas.read_file(f'{DATA_URL}/GeoJSON_assets/India_statewise.geojson')

In [4]:
India_statewise['state']

## Set map projection and summarize GeoJSON data

Map projections project the surface of the earth or a portion of it on a flat surface, such as a computer screen. Map projections approximate the earth's spherical shape (3D) onto a plane (2D). 

***Note: The use of projections doesn't mean that the earth is flat, but, on the contrary.***

The coordinate reference system (CRS) is used to define, using coordinates, the relationship between the two-dimensional, projected map in your GIS and the real places on the earth. The specific type of map projection and coordinate reference system to use, depends on the regional extent of the area you want to work in, on the analysis you want to do and often on the availability of data.[1](https://docs.qgis.org/3.10/en/docs/gentle_gis_introduction/coordinate_reference_systems.html)

In this notebook, the EPSG:3395 CRS is used. It uses the elliptical version of the Marcator projection, with metre (m) as the measurement unit and [Greenwich](https://en.wikipedia.org/wiki/Greenwich) as the prime meridian. This system is useful for very small scale mapping, for parts of the world between 80°S and 84°N. This system excludes the polar areas.[2](https://epsg.io/3395)

In [5]:
India_statewise = India_statewise.to_crs("EPSG:3395")
India_statewise.head()

## Fix naming issues with Indian states

In [6]:
def apply_corrections(input_df):
  for state in list(input_df['state'].values):
    input_df.loc[input_df['state']==state,'state']=re.sub('[^A-Za-z ]+', '',str(state))
  input_df.loc[input_df['state']=='Karanataka','state']='Karnataka' 
  input_df.loc[input_df['state']=='Himanchal Pradesh','state']='Himachal Pradesh' 
  input_df.loc[input_df['state']=='Telengana','state']='Telangana'  
  input_df.loc[input_df['state']=='Dadra and Nagar Haveli','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Dadar Nagar Haveli','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Dadra Nagar Haveli','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Daman & Diu','state']='Dadra and Nagar Haveli and Daman and Diu'
  input_df.loc[input_df['state']=='Daman and Diu','state']='Dadra and Nagar Haveli and Daman and Diu'
  return input_df

In [7]:
India_statewise=apply_corrections(India_statewise)
India_statewise.head()

# Read statewise COVID19 data

In [8]:
covid19_data=pd.read_csv(f'{DATA_URL}/Coronavirus_stats/India/COVID19_India_statewise.csv')

In [9]:
covid19_data=apply_corrections(covid19_data)
covid19_data.head()

In [10]:
noCOVID19_list = list(set(list(India_statewise.state.values)) -set(list(covid19_data.state)))
print('A total of: {} states with no reports of COVID19 ...'.format(len(noCOVID19_list)))
if len(noCOVID19_list)>=1:
  print('\nStates in India with no COVID19 reports:')  
  for noCOVID19_state in noCOVID19_list:
    print('\n{} ...'.format(noCOVID19_state))

# Combine geographical and COVID19 data


The function: ``` covid19_json(covid_df, geo_df) ``` combines the COVID19 dataframe and the GeoPandas dataframe. The output is a dictionary that returns: ```{'json_data': json_data, 'data_frame': merged_df}```

In [11]:
def covid19_json(covid_df, geo_df,verbose=False):
    merged_df = pd.merge(geo_df, covid_df, on='state', how='left')

    try:
      merged_df = merged_df.fillna(0)
    except:
      merged_df.fillna({'total_cases': 0}, inplace=True)
      merged_df.fillna({'deaths': 0}, inplace=True)
      merged_df.fillna({'discharged': 0}, inplace=True)
      if verbose:
        print('Consider updating GeoPandas library ...')
    
    merged_json = json.loads(merged_df.to_json())
    json_data = json.dumps(merged_json)
    return {'json_data': json_data, 'data_frame': merged_df}

In [12]:
merged_data = covid19_json(covid19_data, India_statewise, verbose=True)
merged_json = merged_data['json_data']

In [13]:
merged_data['data_frame'].plot()

# Visualization of statewise COVID19 outbreak in India 

## Helper functions for plotting statewise COVID19 data

In [38]:
def CustomPalette(palette_type, enable_colorInverse=True):
  if (palette_type.lower()=='OrRd'.lower()) or (palette_type.lower()=='reds'):
    palette = OrRd[9]
  elif (palette_type.lower()=='YlGn'.lower()) or (palette_type.lower()=='greens'):
    palette = YlGn[9]
  else:
    palette = brewer['Oranges']
    
  if enable_colorInverse:
    palette = palette[::-1]
  else:
    palette = palette[::1]
  return palette

def CustomHoverTool(advanced_hoverTool, custom_hoverTool, performance_hoverTool, perfstats_hovertool):
  advancedStats_hover=HoverTool(tooltips =
    """<strong><font face="Arial" size="2">@state</font></strong> <br>
       <hr>
       <strong><font face="Arial" size="2">Forecast</font></strong> <br>
       <font face="Arial" size="2">Reported cases: <strong>@total_cases{}</strong></font>
       <font face="Arial" size="2"><p style="color:red; margin:0">+1 day: <strong>@preds_cases{} (±@preds_cases_std{})</strong></p></font>
       <font face="Arial" size="2"><p style="color:green; margin:0">+3 days: <strong>@preds_cases_3{} (±@preds_cases_3_std{})</strong></p></font>
       <font face="Arial" size="2"><p style="color:blue; margin:0">+7 days: <strong>@preds_cases_7{} (±@preds_cases_7_std{})</strong></p></font>
       <hr>  
       <strong><font face="Arial" size="1">Data updated on: {}</font></strong> <br>
       <strong><font face="Arial" size="1">Forecasts updated on: {}</font></strong> <br>
       <strong><font face="Arial" size="1">Forecasts by: https://moad.computer</font></strong> <br>
                                             """.format('{(0,0)}', 
                                                        '{(0,0)}', 
                                                        '{(0,0)}', 
                                                        '{(0,0)}', 
                                                        '{(0,0)}', 
                                                        '{(0,0)}', 
                                                        '{(0,0)}', 
                                                        DATA_UPDATE_DATE,
                                                        FORECASTS_UPDATE_DATE))


  performanceStats_hover=HoverTool(tooltips =
    """<strong><font face="Arial" size="2">@state</font></strong> <br>
       <hr>
       <strong><font face="Arial" size="2">MAPE</font></strong><br>
       <strong><font face="Arial" size="1">(Mean Absolute Percentage Error)</font></strong>
       <font face="Arial" size="2"><p style="color:red; margin:0">+1 day: <strong>@MAPE{}</strong></p></font>
       <font face="Arial" size="2"><p style="color:green; margin:0">+3 days: <strong>@MAPE_3{}</strong></p></font>
       <font face="Arial" size="2"><p style="color:blue; margin:0">+7 days: <strong>@MAPE_7{}</strong></p></font>
       <hr>  
       <strong><font face="Arial" size="1">Data updated on: {}</font></strong><br> 
       <strong><font face="Arial" size="1">Forecasts updated on: {}</font></strong> <br>
       <strong><font face="Arial" size="1">Forecasts by: https://moad.computer</font></strong>                                                    
                                              """.format('{(0.000)}', 
                                                         '{(0.000)}', 
                                                         '{(0.000)}',
                                                         DATA_UPDATE_DATE,
                                                         FORECASTS_UPDATE_DATE))

  simpleStats_hover=HoverTool(tooltips =
    """<strong><font face="Arial" size="3">@state</font></strong> <br>
       <font face="Arial" size="3">Cases: @total_cases{}</font><br>
       <font face="Arial" size="3">Deaths: @deaths{} </font>
       <hr>  
       <strong><font face="Arial" size="1">Updated on: {}</font></strong><br> 
       <strong><font face="Arial" size="1">Data from: https://mohfw.gov.in </font></strong>                                               
                                        """.format('{(0,0)}', 
                                                   '{(0,0)}',
                                                   DATA_UPDATE_DATE))

  perfStats_hover=HoverTool(tooltips =
    """<strong><font face="Arial" size="3">@state</font></strong> <br>
       <font face="Arial" size="3">Cases: @total_cases{}</font><br>
       <font face="Arial" size="3">Deaths: @deaths{} </font>
       <hr>  
       <strong><font face="Arial" size="1">Data updated on: {}</font></strong><br> 
       <strong><font face="Arial" size="1">Forecasts updated on: {}</font></strong><br>
       <strong><font face="Arial" size="1">Data from: https://mohfw.gov.in </font></strong>                                               
                                        """.format('{(0,0)}', 
                                                   '{(0,0)}',
                                                   DATA_UPDATE_DATE,
                                                   FORECASTS_UPDATE_DATE))

  standard_hover = HoverTool(tooltips = [('State','@state'),
                                         ('Cases', '@total_cases'),
                                         #('Discharged/migrated', '@discharged'),
                                         ('Deaths', '@deaths')])
  
  if performance_hoverTool:
    hover  = performanceStats_hover
  elif advanced_hoverTool:
    hover = advancedStats_hover
  elif custom_hoverTool:
    hover  = simpleStats_hover
  elif perfstats_hovertool:
    hover = perfStats_hover
  else:
    hover = standard_hover
  return hover

def MapOverlayFormatter(map_overlay):
  if map_overlay:
    xmin, xmax = 7570000, 10950000
    ymin, ymax = 890000, 4850000
    return xmin, xmax, ymin, ymax

def geographic_overlay(plt, geosourceJson=None, colorBar=None,
                       colorMapper=None, colorMode='', hoverTool=None,
                       mapOverlay=True, enableTapTool=False, enableToolbar=True):
  if mapOverlay:
    wmts = WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png")
    plt.add_tile(wmts)
    plt.xaxis.axis_label = 'longitude'
    plt.yaxis.axis_label = 'latitude'
  
  plt.xgrid.grid_line_color = None
  plt.ygrid.grid_line_color = None
  plt.axis.visible = False
  plt.patches('xs','ys', 
              source = geosourceJson, 
              fill_color = {'field' : colorMode, 
                            'transform' : colorMapper},
              line_color = 'purple', 
              line_width = 0.5, 
              fill_alpha = 0.60 if enableTapTool else 0.65,
              nonselection_alpha = 0.65)
  plt.add_layout(colorBar, 'right')
  plt.add_tools(hoverTool)
  if enableTapTool:
    plt.add_tools(TapTool())
  if enableToolbar:
    plt.toolbar.autohide = True
  if plt.title is not None:
    plt.title.text_font_size = '30pt'
  return plt

def lakshadweep_correction(plt, input_df=None, advanced_plotting=False):
  if advanced_plotting:
    source = ColumnDataSource(data=dict(
        x=[8075000], y=[1250000], state=['Lakshadweep'],
        total_cases=[input_df.loc[input_df['state']=='Lakshadweep','total_cases']],
        deaths=[input_df.loc[input_df['state']=='Lakshadweep','deaths']],
        preds_cases=[input_df.loc[input_df['state']=='Lakshadweep','preds_cases']],
        preds_cases_std=[input_df.loc[input_df['state']=='Lakshadweep','preds_cases_std']],
        MAPE=[input_df.loc[input_df['state']=='Lakshadweep','MAPE']],
        preds_cases_3=[input_df.loc[input_df['state']=='Lakshadweep','preds_cases_3']],
        preds_cases_3_std=[input_df.loc[input_df['state']=='Lakshadweep','preds_cases_3_std']],
        MAPE_3=[input_df.loc[input_df['state']=='Lakshadweep','MAPE_3']],
        preds_cases_7=[input_df.loc[input_df['state']=='Lakshadweep','preds_cases_7']],
        preds_cases_7_std=[input_df.loc[input_df['state']=='Lakshadweep','preds_cases_7_std']],
        MAPE_7=[input_df.loc[input_df['state']=='Lakshadweep','MAPE_7']]
                                      ))
  else:
    source = ColumnDataSource(data=dict(
        x=[8075000], y=[1250000], state=['Lakshadweep'],
        total_cases=[input_df.loc[input_df['state']=='Lakshadweep','total_cases']],
        deaths=[input_df.loc[input_df['state']=='Lakshadweep','deaths']]))

  plt.circle(x='x', y='y', size=25, source=source,
             line_color='purple', color='blue',
             fill_alpha=0.075, nonselection_alpha=0.20)
  return plt

def CustomTitleFormatter():
  xtext, ytext = 8350000, 4425000
  xbox, ybox = 9400000, 4575000
  return xtext, ytext, xbox, ybox

def CustomTitleOverlay(plt, xtext=0, ytext=0,
                       xbox=0, ybox=0, input_df=None, 
                       advanced_plotting=False):
  
  overlayText=Label(x=xtext, y=ytext, 
                    text="COVID19 in India",
                    text_font_size='25pt')
    
  plt.add_layout(overlayText) 

  source = ColumnDataSource(data=dict(x=[xbox], y=[ybox], state=['India'],
                                      total_cases=[input_df['total_cases'].sum()],
                                      deaths=[input_df['deaths'].sum()]))

  plt.rect(x='x', y='y', width=2250000, height=250000, 
           color="#CAB2D6", source=source, line_color='purple',
           #width_units='screen', height_units='screen',
           fill_alpha=0.25)
  return plt

    
def covid19_plot(covid19_geosource, input_df=None, input_field=None,
                 color_field='total_cases', plot_title=None,
                 map_overlay=True, palette_type='OrRd', integer_plot=False,
                 custom_hovertool=True, enable_LakshadweepStats=True,
                 enable_IndiaStats=False, enable_advancedStats=False,
                 enable_performanceStats=False, enable_foecastPerf=False,
                 enable_toolbar=False):
  
  palette = CustomPalette(palette_type, enable_colorInverse=False \
                          if enable_performanceStats else True)
  color_mapper = LinearColorMapper(palette=palette, low=0, 
                                   high=int(10*(np.ceil(np.max(
                                       input_df[color_field].values)/10)))\
                                        if not enable_performanceStats else np.round(
                                       (np.max(input_df[color_field].values)),3)) 
  if integer_plot:
    format_tick=NumeralTickFormatter(format='0,0')
  else:
    format_tick=NumeralTickFormatter(format=str(input_df[input_field].values.astype('int')) \
                                     if not enable_performanceStats else\
                                     str(np.round(
        (input_df[input_field].values.astype('float')),1)))
  color_bar = ColorBar(color_mapper=color_mapper, 
                       label_standoff=14, 
                       formatter=format_tick,
                       border_line_color=None, 
                       major_label_text_font_size='12px',
                       location = (0, 0))
  xmin,xmax,ymin,ymax=MapOverlayFormatter(map_overlay)
  hover=CustomHoverTool(enable_advancedStats,custom_hovertool,
                        enable_performanceStats,enable_foecastPerf)

  plt=figure(title = plot_title,
             x_range=(xmin, xmax) if map_overlay else None,
             y_range=(ymin, ymax) if map_overlay else None,
             tools='save' if enable_toolbar else '', 
             plot_height = 530, plot_width = 530,
             toolbar_location = 'left' if enable_toolbar else None,
             lod_factor=int(1e7), lod_threshold=int(2),
             # output_backend="webgl"
            ) 
        
  plt=geographic_overlay(plt, geosourceJson=covid19_geosource,
                         colorBar=color_bar, colorMapper=color_mapper, colorMode=input_field,
                         hoverTool=hover, mapOverlay=map_overlay, enableToolbar=enable_toolbar,
                         enableTapTool=True if (
                             (enable_advancedStats) or (enable_performanceStats)) \
                         else False)
  
  if enable_LakshadweepStats:
    plt=lakshadweep_correction(plt, input_df=input_df, 
                               advanced_plotting=True if (
                                   (enable_advancedStats) or (enable_performanceStats)) \
                               else False)

  if enable_IndiaStats:
    xtext,ytext,xbox,ybox=CustomTitleFormatter()
    plt=CustomTitleOverlay(plt, xtext=xtext, ytext=ytext,
                           xbox=xbox, ybox=ybox, input_df=input_df, 
                           advanced_plotting=True if (
                               (enable_advancedStats) or (enable_performanceStats)) \
                           else False)
  plt.xaxis.major_tick_line_color=None  
  plt.yaxis.major_tick_line_color=None
  plt.xaxis.minor_tick_line_color=None 
  plt.yaxis.minor_tick_line_color=None 
  plt.xaxis[0].ticker.num_minor_ticks=0
  plt.yaxis[0].ticker.num_minor_ticks=0
  plt.yaxis.formatter=NumeralTickFormatter(format='0,0')
  return plt

## Create interactive plot using Bokeh

In [39]:
DATA_UPDATE_DATE='26-November-2021'
FORECASTS_UPDATE_DATE='26-November-2021'

covid19_geosource=GeoJSONDataSource(geojson=merged_json)
plot_title='COVID19 outbreak in India'
app_title='COVID19 India'

India_totalCases=covid19_data['total_cases'].sum()
India_totalDeaths=covid19_data['deaths'].sum()
print(India_totalCases)

basic_covid19_plot = covid19_plot(covid19_geosource, 
                                  input_df=covid19_data,
                                  input_field='total_cases',
                                  color_field='total_cases',
                                  enable_IndiaStats=True,
                                  integer_plot=True,
                                  plot_title=plot_title)

curdoc().title=app_title
basic_covid19_layout = column(basic_covid19_plot)
curdoc().add_root(basic_covid19_layout)

## Display the interactive plot

In [40]:
output_notebook()
show(basic_covid19_layout)